In [61]:
import spacy
import json
import os
from nltk.corpus import stopwords, words
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions
import string

In [168]:
files = []
subdir = tuple(os.walk("../MergedData/merged_movie_jsons/2022"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [169]:
corpus = []
my_pos = set(["ADJ"])
nlp = spacy.load("en_core_web_lg") #python -m spacy download en_core_web_lg


for file in files:
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        s = nlp(review)
        clean = []
        for word in s:
            if word.pos_ in my_pos:
                clean.append(str(word.lemma_))
        corpus.append(" ".join(clean))
    break

In [170]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations
# print(type(stopwords.words("english")))

In [171]:
with open("../stopwords.json", "r") as f:
    common = json.load(f)
    common.extend(stopwords.words("english"))
    common = set(common)


eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word not in common and word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [172]:
proc_data = [clean(movie_revs).split() for movie_revs in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [173]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict,  normalize=True, dictionary=input_dict, slope=2.5, smartirs="Lfc")
vector = model[input_corpus]

In [174]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f

In [175]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [176]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df["TF-IDF"].between(0.8 * df["TF-IDF"].median(), 1.05 * df["TF-IDF"].median(), inclusive=True))
    # pass

In [177]:
# kmeans = KMeans(n_clusters=100, init="k-means++", n_init=10000)
# vect = TfidfVectorizer(stop_words="english")
# features = vect.fit_transform([ " ".join(data) for data in proc_data])
# kmeans.fit(features)

In [178]:
# df2 = pd.DataFrame(corpus)
# df2["clusters"] = kmeans.labels_
# df2

In [179]:
# print("Cluster centrojds: \n")
# order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vect.get_feature_names_out()
# for i in range (100):
#     print("cluster %d:" % i)
#     for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
#         print(' %s ' % terms [j])

In [184]:
med = df["TF-IDF"].median()
mean = df["TF-IDF"].mean()
vibes = set()
for k, v in d.items():
    if 0.95 *  med <= v <= 1.05 * med:
        vibes.add(k)


uninvited
mystic
relatable
shallow
tiny
wholesome
unhappy
peripheral
lead
potential
wealthy
bold
minor
compelling
successful
additional
wrong
melancholic
trashy
dimensional
unstoppable
vampiric
dismal
reliable
unnecessary
stiff
upset
suspicious
hopeful
adorable
sexy
generic
superb
brutal
unfamiliar
wide
future
weekly
faithful
experienced
disappointed
comfortable
social
unconditional
unknown
likable
parallel
precious
outcast
principal
jaded
international
authentic
vampire
amateur
suspenseful
logical
